In [1]:
import torch
import torch.nn as nn

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

SOS = 0
EOS = 1

In [10]:
# dict형태로 단어집합 만들기
class Vocab:
    def __init__(self):
        self.vocab2idx = {'<SOS>':SOS, '<EOS>':EOS}
        self.idx2vocab = {SOS:'<SOS>', EOS:'<EOS>'}
        self.vocab_cnt = {}
        self.n_vocab = len(self.vocab2idx)
    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            
            # key, value 둘 모두 word가 될 수 있다
            if word not in self.vocab2idx:
                
                # 새로운 단어를 발견하면 vocab2idx 맨 뒤에 추가
                self.vocab2idx[word] = self.n_vocab
                
                # 새로운 단어를 발견하면 cnt는 1, 같은 단어 발견하면 1씩 추가
                self.vocab_cnt[word] = 1
                
                # vocab2idx 거꾸로만 하면 됨
                self.idx2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_cnt[word] += 1

                
# 최대 길이가 넘는 문장은 버리기                
def filter_pair(pair, source_max_length, target_max_length):
    # pair[0] == encoder 입력 문장 == source input
    # pair[1] == decoder 입력 문장 == target input
    return len(pair[0].split(' ')) < source_max_length and len(pair[1].split(' ') < target_max_length)

# 분절 + 소문자화 + 인코더 입력 데이터 / 디코더 입력 데이터 분리
def preprocess(corpus, source_max_length, target_max_length):
    pairs = []
    for line in corpus:
        # X, Y 분절 + 소문자화
        splower = [s for s in line.strip().lower().split('\t')]
        pairs.append(splower)
    
    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    
    source_vocab = Vocab()
    target_vocab = Vocab()
    
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    
    # pairs는결국 source_vocab과 target_vocab의 합집합이다
    return pairs, source_vocab, target_vocab

In [12]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, m):
        super().__init__()
        self.units = units
        self.embedding = nn.Embedding(vocab_size, m)
    
        units = m # 뉴런개수와 input_dim이 우연히 같을 뿐 혼동 ㄴㄴ
        self.lstm = nn.LSTM(m, units)
        
    def forward(self, inputs, h0c0): # h0c0 == (h0, c0)
        inputs = self.embedding(inputs).view(1, 1, -1) # flatten ??
        output, h0c0 = self.lstm(inputs, h0c0)
        return output, h0c0

class Decoder(nn.Module):
    def __init__(self, vocab_size, m): # 인코더랑 디코더 vocab_size는 다를 수 있다
        super().__init__()
        self.m = m
        self.embedding = nn.Embedding(vocab_size, m)
        units = m # 이것도 우연임
        self.lstm = nn.LSTM(m, units)
        self.out = nn.Linear(m, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, inputs, h0c0):
        inputs = self.embedding(inputs).view(1, 1, -1)
        output, h0c0 = self.lstm(inputs, h0c0)
        y = self.softmax(self.out(output[0])) # N=1개씩 step을 계산하기 위해 output[0]
        return y, h0c0